In [1]:
import pandas as pd
import numpy as np
import pickle

import os
from dotenv import load_dotenv

load_dotenv()
# API_KEY = os.getenv("API_KEY")
VOYAGE_API_KEY = os.getenv("VOYAGE_API_KEY")

In [2]:
user_train_path = './train_valid_test/user_train.pkl'
user_valid_path = './train_valid_test/user_valid.pkl'
user_test_path = './train_valid_test/user_test.pkl'

In [3]:
with open(user_train_path, 'rb') as f:
    user_train_dict = pickle.load(f)

In [4]:
with open(user_valid_path, 'rb') as f:
    user_valid_dict = pickle.load(f)

In [5]:
with open(user_test_path, 'rb') as f:
    user_test_dict = pickle.load(f)

In [6]:
ratings_df = pd.read_csv('ratings.csv')
ratings_df.head()

,user_id,movie_id,rating
0,1,3186,4
1,1,1721,4
2,1,1022,5
3,1,1270,5
4,1,2340,3


In [7]:
user_id = 1
print(user_train_dict[user_id])
print(user_valid_dict[user_id])
print(user_test_dict[user_id])

[[1961, 2028, 3105, 938, 1962, 1035, 150, 2018, 1028, 1097, 914, 2797, 1287, 2762, 1246, 661, 2918, 531, 3114, 2791, 1029, 2321, 1197, 594, 2398, 1545, 527, 1, 588, 2687, 745, 595, 2294, 783, 2355, 1907]]
[1566]
[48]


In [8]:
movies_df = pd.read_csv('movies.csv')
movies_df.head()

,Unnamed: 0,movie_id,name,cleaned_genre,year
0,0,1.0,Toy Story,"Animation,Children's,Comedy",1995.0
1,1,2.0,Jumanji,"Adventure,Children's,Fantasy",1995.0
2,2,3.0,Grumpier Old Men,"Comedy,Romance",1995.0
3,3,4.0,Waiting to Exhale,"Comedy,Drama",1995.0
4,4,5.0,Father of the Bride Part II,Comedy,1995.0


In [9]:
movies_summary_df = pd.read_csv('movie_summary_25_words.csv')
movies_summary_df.head()

,movie_id,name,cleaned_genre,year,summary
0,1,Toy Story,"Animation,Children's,Comedy",1995,Toy Story is a groundbreaking animated film th...
1,2,Jumanji,"Adventure,Children's,Fantasy",1995,"""Jumanji (1995) is a thrilling adventure film ..."
2,3,Grumpier Old Men,"Comedy,Romance",1995,"""Grumpier Old Men is a hilarious and heartwarm..."
3,4,Waiting to Exhale,"Comedy,Drama",1995,"""Waiting to Exhale"" (1995) is a heartfelt dram..."
4,5,Father of the Bride Part II,Comedy,1995,Father of the Bride Part II (1995) is a heartw...


In [10]:
movies_summary_df.shape

(3883, 5)

In [11]:
from collections import defaultdict
from datetime import datetime
import pandas as pd
import json
import numpy as np
import openai
import requests
from tenacity import retry, wait_random_exponential, stop_after_attempt

In [12]:
test_text = movies_summary_df.iloc[1873]['summary'] + " Genre: " + movies_summary_df.iloc[1873]['cleaned_genre']
print(test_text)

"All the King's Men" (1949) is a gripping political drama that explores corruption and power through the rise and fall of a charismatic Southern politician. Genre: Drama


In [13]:
movies_summary_df['all_text'] = movies_summary_df.apply(lambda x: x['summary'] + " Genre: " + x['cleaned_genre'], axis = 1)
movies_summary_df.head()

,movie_id,name,cleaned_genre,year,summary,all_text
0,1,Toy Story,"Animation,Children's,Comedy",1995,Toy Story is a groundbreaking animated film th...,Toy Story is a groundbreaking animated film th...
1,2,Jumanji,"Adventure,Children's,Fantasy",1995,"""Jumanji (1995) is a thrilling adventure film ...","""Jumanji (1995) is a thrilling adventure film ..."
2,3,Grumpier Old Men,"Comedy,Romance",1995,"""Grumpier Old Men is a hilarious and heartwarm...","""Grumpier Old Men is a hilarious and heartwarm..."
3,4,Waiting to Exhale,"Comedy,Drama",1995,"""Waiting to Exhale"" (1995) is a heartfelt dram...","""Waiting to Exhale"" (1995) is a heartfelt dram..."
4,5,Father of the Bride Part II,Comedy,1995,Father of the Bride Part II (1995) is a heartw...,Father of the Bride Part II (1995) is a heartw...


### Using SFR-Embedding-Mistral for generating embeddings (Best on METB Leaderboard)
https://huggingface.co/Salesforce/SFR-Embedding-Mistral

In [17]:
import torch
import torch.nn.functional as F
from torch import Tensor
from transformers import AutoTokenizer, AutoModel
def last_token_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    left_padding = (attention_mask[:, -1].sum() == attention_mask.shape[0])
    if left_padding:
        return last_hidden_states[:, -1]
    else:
        sequence_lengths = attention_mask.sum(dim=1) - 1
        batch_size = last_hidden_states.shape[0]
        return last_hidden_states[torch.arange(batch_size, device=last_hidden_states.device), sequence_lengths]

In [18]:
tokenizer = AutoTokenizer.from_pretrained('Salesforce/SFR-Embedding-Mistral')
model = AutoModel.from_pretrained('Salesforce/SFR-Embedding-Mistral')
tokenizer.add_eos_token = True

tokenizer_config.json:   0%|          | 0.00/956 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/22.2k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.28G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [23]:
import time

max_length = 4096
input_texts = test_text

start = time.time()
batch_dict = tokenizer(input_texts, max_length=max_length - 1, padding=True, truncation=True, return_tensors="pt")
outputs = model(**batch_dict)
embeddings = last_token_pool(outputs.last_hidden_state, batch_dict['attention_mask'])
print("Time Taken:", time.time() - start)
# normalize embeddings
embeddings = F.normalize(embeddings, p=2, dim=1)
print(len(embeddings[0]))

Time Taken: 5.457363605499268
4096


### Using Voyage-2 for generating embeddings (Better than openai text-embedding-large)
https://docs.voyageai.com/embeddings/#python-api

In [33]:
import voyageai
import time

vo = voyageai.Client(api_key = VOYAGE_API_KEY)
texts = [test_text]
start = time.time()
result1 = vo.embed(texts, model="voyage-lite-02-instruct", input_type="document")
print("Time taken: {}".format(time.time() - start))
# print(len(result.embeddings[0]))

Time taken: 0.4309725761413574


### Generating embeddings for movies

In [14]:
movies_summary_df.head()

,movie_id,name,cleaned_genre,year,summary,all_text
0,1,Toy Story,"Animation,Children's,Comedy",1995,Toy Story is a groundbreaking animated film th...,Toy Story is a groundbreaking animated film th...
1,2,Jumanji,"Adventure,Children's,Fantasy",1995,"""Jumanji (1995) is a thrilling adventure film ...","""Jumanji (1995) is a thrilling adventure film ..."
2,3,Grumpier Old Men,"Comedy,Romance",1995,"""Grumpier Old Men is a hilarious and heartwarm...","""Grumpier Old Men is a hilarious and heartwarm..."
3,4,Waiting to Exhale,"Comedy,Drama",1995,"""Waiting to Exhale"" (1995) is a heartfelt dram...","""Waiting to Exhale"" (1995) is a heartfelt dram..."
4,5,Father of the Bride Part II,Comedy,1995,Father of the Bride Part II (1995) is a heartw...,Father of the Bride Part II (1995) is a heartw...


In [17]:
import voyageai
vo = voyageai.Client(api_key = VOYAGE_API_KEY)
cnt = 0
def getEmbeddings(text):
    texts = [text]
    res = vo.embed(texts, model="voyage-lite-02-instruct")
    return res.embeddings

In [18]:
movies_summary_df['embeddings'] = movies_summary_df.apply(lambda x: getEmbeddings(x['all_text']), axis = 1)

ServiceUnavailableError: The server is overloaded or not ready yet.